# RNN Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers.experimental.preprocessing import Normalization

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense ,Dropout, Bidirectional

from sklearn import set_config

set_config(display='diagram')

In [2]:
df = pd.read_csv('../wildfire_prediction/data/merged_file.csv', index_col=0)

In [3]:
#Replacing na by 0
df.Estimated_fire_area.fillna(0, inplace=True)
df.Mean_estimated_fire_brightness.fillna(0, inplace=True)
df.Mean_estimated_fire_radiative_power.fillna(0, inplace=True)

#Defining features and target
X = df.drop(columns=['Estimated_fire_area', 'Date_x'])
y = df.Estimated_fire_area

#Region Encoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded = encoder.fit_transform(X[['Region']])
encoder.categories_
for i, col in enumerate(encoder.categories_[0]):
    X[col] = pd.DataFrame(encoded)[i]
X.drop(columns='Region', inplace=True)

#Imputing null values
numerical_features = X.dtypes[(df.dtypes == 'float64') | (X.dtypes == 'int64')].index
imputer = KNNImputer()
values = imputer.fit_transform(X[X.dtypes[(df.dtypes == 'float64') | (X.dtypes == 'int64')].index])
X[X.dtypes[(df.dtypes == 'float64') | (X.dtypes == 'int64')].index] = values

#Train Test Split
len_ = int(0.8*df.shape[0])
X_train = X[:len_]
X_test = X[len_:]
y_train = y[:len_]
y_test = y[len_:]

In [4]:
X_train.shape

(32776, 53)

In [5]:
y_test.shape

(8195,)

In [8]:
X_train = X_train.to_numpy().reshape((32776, 53, 1))
X_test = X_test.to_numpy().reshape((8195, 53, 1))
y_test = y_test.to_numpy().reshape((8195,1))

In [9]:
# regressor = Sequential()
# regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (X_train.shape[1],1) ) ))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units= 30 , return_sequences=True))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units= 30 , return_sequences=True))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units= 30))
# regressor.add(Dropout(0.2))
# regressor.add(Dense(units = 1,activation='linear'))
# regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
# regressor.fit(X_train, y_train, epochs=10,batch_size=32 )

In [10]:
# y_pred = regressor.predict(X_test)

In [11]:
# np.abs(y_pred - y_test).mean()

In [12]:
# model.evaluate(X_test, y_test)

In [13]:
# model = Sequential()
# model.add(layers.SimpleRNN(units=2, activation='tanh'))
# model.add(layers.Dense(1, activation="linear"))

# # The compilation
# model.compile(loss='mse', 
#               optimizer='rmsprop')  # Recommanded optimizer for RNN
# # The fit
# model.fit(X_train, y_train,
#          batch_size=16,
#          epochs=10, verbose=0)


In [14]:
# model.evaluate(X_test, y_test)

In [15]:
# mse = np.square(np.subtract(y_test,model.predict(X_test))).mean()
# rmse = np.sqrt(mse)
# rmse

In [16]:
# average_y = y_test.mean()

In [17]:
# print('Baseline')
# print('mse', np.square(np.subtract(y_test,average_y)).mean())
# print('rmse', np.sqrt(np.square(np.subtract(y_test,average_y)).mean()))

In [31]:
def init_model():
    
#     model = Sequential()
#     model.add(layers.LSTM(20, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],1)))
#     model.add(layers.LSTM(10, return_sequences=True, activation='tanh'))
#     model.add(layers.Dense(5, activation='relu'))
#     model.add(layers.Dense(1, activation='linear'))
    

    
    opt = RMSprop(learning_rate=0.005)
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = Sequential()
#     model.add(normalizer)
    model.add(layers.LSTM(200, return_sequences=True, activation='tanh', input_shape=(X_train.shape[1],1)))
    model.add(layers.LSTM(100, activation='tanh'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer='rmsprop', 
                  metrics=['mae'])
#     model.compile(loss='mse', 
#                   optimizer=RMSprop(learning_rate=0.0003), 
#                   metrics=['mae'])
    
    return model

In [32]:
model = init_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 53, 200)           161600    
_________________________________________________________________
lstm_19 (LSTM)               (None, 100)               120400    
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 505       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 6         
Total params: 282,511
Trainable params: 282,511
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', verbose=1, patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train,
            validation_split=0.2,
            epochs=500, 
            batch_size=64,
            callbacks=[es], verbose=0)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])

In [ ]:
res = model.evaluate(X_test, y_test, verbose=0)

print(f'mae on the test set : {res[1]:.0f}')

In [ ]:
res